In [1]:
# if you need to install Pybullet:
#   pip3 install Pybullet   (python3)
#   pip install PyBullet    (python2.x)

# let's start with imports
import numpy as np
from numpy.random import seed
from numpy.random import rand

from engine import *

# seed random number generator
seed(1)

pybullet build time: Nov 28 2023 23:51:11


In [2]:
# create a pybullet engine from the engine module
engine = PyBulletEnvironment()


In [3]:
#load shovel
startPos = [0,0,0]
startOrientation = p.getQuaternionFromEuler([0,0,0])
robot_urdf = './urdf/shovel/shovelFlat.urdf'

# load the robot
engine.open_environment(robot_urdf, startPos, startOrientation)

MESA: warning: Driver does not support the 0xa7a0 PCI ID.

startThreads creating 1 threads.
starting thread 0



libGL error: failed to create dri screen
libGL error: failed to load driver: iris
MESA: warning: Driver does not support the 0xa7a0 PCI ID.
libGL error: failed to create dri screen
libGL error: failed to load driver: iris


started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa/X.org
GL_RENDERER=llvmpipe (LLVM 12.0.0, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 21.2.6
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 21.2.6
Vendor = Mesa/X.org
Renderer = llvmpipe (LLVM 12.0.0, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Mesa/X.org
ven = Mesa/X.org


In [4]:
# load pebbles

# get shovel_link position
shovelLinkState = p.getLinkState(engine.ID[1], 0)

# admissible init positions
minpos = np.asarray(shovelLinkState[0]) - 3*1e-1
maxpos = np.asarray(shovelLinkState[0]) + 3*1e-1

# number of pebbles
pebbleNum = 50

# store pebbles IDs
pebbleId = np.zeros(pebbleNum)

# scatter pebbles
for i in range (pebbleNum):
    
    # generate init pos
    startPos = minpos + (rand(3) * (maxpos - minpos))
    startPos[-1] = 0.2
    
    # set orientation
    startOrientation = p.getQuaternionFromEuler([0,0,0])
    
    # load pebble 
    engine.load_urdf('./urdf/pebbles/pebbles.urdf', startPos, startOrientation)    
    
# simulate
engine.simulate(2)

In [5]:
# here we control the shovel to move to a target position and orientation using velocity control
# the outer loop alternates between controlling the position and the orientation
# the inner loop controls the velocity of the joint until the target position is reached

# in the next params we define the control mode, the max force on the joint, the target position and velocity

# max force on joint
maxForce = 1e3

# target pos and vel - prismatic
pos_std = 0.3
targetPosMean = np.mean(p.getJointInfo(engine.ID[1], 0)[8:10])
targetPosStd =  np.min(np.append(np.array(np.abs(p.getJointInfo(engine.ID[1], 0)[8:10])),pos_std))
targetVelPos = 0.2

# target pos and vel - revolute
ang_std = 0.5
targetAngMean = np.mean(p.getJointInfo(engine.ID[1], 1)[8:10])
targetAngStd =  np.min(np.append(np.array(np.abs(p.getJointInfo(engine.ID[1], 1)[8:10])),ang_std))
targetVelAng = 1

# current control mode
mode = p.VELOCITY_CONTROL

# init target Pos and Vel    
targetPos= targetPosMean+targetPosStd
targetAng= targetAngMean+targetAngStd

# how many iterations we want to run
Niter = 5

# accuracy threshold
threshold = 5e-2

# control outer loop
for i in range(Niter):        
    
    # error init - Pos/Ang    
    ePos = engine.compute_joint_error(0,targetPos)        
    eAng = engine.compute_joint_error(1,targetAng)
    
    # init stop flags
    reach_pos = 0
    reach_ang = 0
    
    # control inner loop
    while reach_pos==0 or reach_ang==0:
        
        # control - Pos
        if np.abs(ePos) > threshold and reach_pos == 0:
            targetVelPosCtrl = targetVelPos*np.sign(ePos)
            engine.control_joint(0, targetVelPosCtrl, mode, maxForce)
        else:
            engine.control_joint(0, 0, mode, maxForce)
            reach_pos = 1
        
        # control - Ang
        if np.abs(eAng) > threshold and reach_ang == 0:
            targetVelAngCtrl = targetVelAng*np.sign(eAng)
            engine.control_joint(1, targetVelAngCtrl, mode, maxForce)
        else:
            engine.control_joint(1, 0, mode, maxForce)
            reach_ang = 1
        
        # simulate
        engine.simulate(0.1)
        
        # error update - Pos/Ang    
        ePos = engine.compute_joint_error(0,targetPos)        
        eAng = engine.compute_joint_error(1,targetAng)
        
    # redefine targets
    targetPos= targetPosMean + (targetPosMean-targetPos)
    targetAng= targetAngMean + (targetAngMean-targetAng)

In [6]:
# disconnect engine
engine.close_environment()

numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
